In [1]:
from dputils.scrape import Scraper, Tag

In [31]:
def get_url_list_from_amazon(url):
    sc = Scraper(url)
    urllist = sc.get_repeating_date_from_page(
        target=Tag('div', cls='s-result-list'),
        items = Tag('div', cls='s-result-item'),
        url = Tag('a', cls='a-link-normal', output='href'),
    )
    return [item['url'] for item in urllist if item['url']]

In [33]:
url = 'https://www.amazon.in/s?k=mobile'
get_url_list_from_amazon(url) 

['https://aax-eu.amazon.in/x/c/JDJWABJCWpHJFnBpKGwolf4AAAGSKYoUNAMAAAH2AQBvbm9fdHhuX2JpZDIgICBvbm9fdHhuX2ltcDEgICALrUY-/https://www.amazon.in/stores/page/D4FF818A-6C20-480D-960F-8ABAA178C71B/?_encoding=UTF8&store_ref=SB_A1034153KHL56M2RAWLC-A016707833EA2OADGJBS6&pd_rd_plhdr=t&aaxitk=067d11349f9fe49292c6a62faa753564&hsa_cr_id=0&lp_asins=B0DDTZF6G1%2CB0DDTXJ2MX%2CB0DDTXW9WH&lp_query=mobile&lp_slot=auto-sparkle-hsa-tetris&ref_=sbx_be_s_sparkle_lsi4d_bkgd&pd_rd_w=rqTcZ&content-id=amzn1.sym.df9fe057-524b-4172-ac34-9a1b3c4e647d%3Aamzn1.sym.df9fe057-524b-4172-ac34-9a1b3c4e647d&pf_rd_p=df9fe057-524b-4172-ac34-9a1b3c4e647d&pf_rd_r=SMH4W0469EVEWCHH78RD&pd_rd_wg=9V3zA&pd_rd_r=3ff3f3d9-e31f-4747-af8f-20d6b4b6451b',
 '/sspa/click?ie=UTF8&spc=MTozNDY4NDQ1NzY0MjQzNDc1OjE3MjcyNzM3Njc6c3BfYXRmOjMwMDE5MzUwNTg4ODgzMjo6MDo6&url=%2FiQOO-Storage-Snapdragon-6000mAh-FlashCharge%2Fdp%2FB07WHPLH66%2Fref%3Dsr_1_1_sspa%3Fdib%3DeyJ2IjoiMSJ9.wQ9T254XRacqS5wcg7MgPBOc4rLQizUoKSvpZbyQc5ulpoaB7XQhiPUHsC08Q6UMLkvv_qlhmP

database setup

In [43]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, Integer, String, Boolean

Base = declarative_base()

class Product(Base):
    __tablename__ = 'products'
    id = Column(Integer, primary_key=True)
    url = Column(String)
    title = Column(String)
    sprice = Column(String)
    oprice = Column(String)
    reviews = Column(String)

engine = create_engine('sqlite:///amazon.db')
Base.metadata.create_all(engine)

def opendb():
    engine = create_engine('sqlite:///amazon.db')
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

In [41]:
def get_details_from_url(product_url):
    if 'html' in product_url:
        return None
    if product_url.startswith('/'):
        base_url = 'https://www.amazon.in'
        product_url = base_url + product_url
    sc = Scraper(product_url)
    product = sc.get_data_from_page(
        name = Tag('h1', id='title'),
        sprice = Tag('span', cls='priceToPay'),
        oprice = Tag('span', cls='a-price a-text-price'),
        reviews = Tag('span', id='acrCustomerReviewText'),
    )
    return product

In [45]:
for product_url in get_url_list_from_amazon(url):
    product = get_details_from_url(product_url)
    if not product:
        continue
    if not product['name']:
        continue
    db = opendb()
    item = Product(
        url = product_url,
        title = product['name'].strip(),
        sprice = product['sprice'].strip(),
        oprice = product['oprice'].strip(),
        reviews = product['reviews'].strip(),
    )
    db.add(item)
    db.commit()
    print("Product details added to database")
db.close()

Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database
Product details added to database


TypeError: 'NoneType' object is not subscriptable